# VacationPy
----

#### Note
* Input csv file was generated from WeatherPy script which contains randomly generated data so changes will be reflected in this script if WeatherPy is before running this script.

In [1]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW\\Python_APIs_and_Sentiment_Analysis\\WeatherPy\\code'

In [2]:
os.chdir(os.path.join('..','..','..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

In [3]:
# Import API key
from api_keys import g_key

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
#Confirming working directory before calling City_Data.csv 
os.chdir(orig_working_directory)
os.chdir(os.path.join('..'))
os.getcwd()

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW\\Python_APIs_and_Sentiment_Analysis\\WeatherPy'

In [6]:
import csv
file="output_data/City_Data.csv"
weather_data=pd.read_csv(file)
weather=weather_data.drop(['Unnamed: 0','Date'],axis=1)
weather.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Conceição do Araguaia,-8.26,-49.26,27.00,78,64,1.50,BR
1,Mataura,-46.19,168.86,9.77,58,46,0.97,NZ
2,Kapaa,22.08,-159.32,27.78,69,40,6.70,US
3,Pevek,69.70,170.31,5.16,69,12,7.86,RU
4,Chokurdakh,70.63,147.92,6.84,60,100,1.98,RU
5,Punta Arenas,-53.15,-70.92,6.00,87,90,4.10,CL
6,Port Elizabeth,-33.92,25.57,10.00,100,0,1.00,ZA
7,Qaanaaq,77.48,-69.36,-2.99,94,11,0.43,GL
8,Tasiilaq,65.61,-37.64,5.00,80,98,6.70,GL
9,Ushuaia,-54.80,-68.30,9.00,66,90,7.20,AR


### Humidity Heatmap
* Configuring gmaps.
* Using the Lat and Lng as locations and Humidity as the weight.
* Adding Heatmap layer to map.

In [7]:
#Importing dependencies
import gmaps
import pandas as pd

In [8]:
#Creating a new DataFrame with essential columns for Humidity heatmap
data=weather[['Lat','Lng','City','Humidity']]
data.head()

,Lat,Lng,City,Humidity
0,-8.26,-49.26,Conceição do Araguaia,78
1,-46.19,168.86,Mataura,58
2,22.08,-159.32,Kapaa,69
3,69.70,170.31,Pevek,69
4,70.63,147.92,Chokurdakh,60


In [9]:
#Grouping location by lat,long and City
lat_lon_grp=data.groupby(['Lat','Lng','City']).count()

#re-assinging lat_long_grp to new variable called marker_data and resetting the index to easily pull out needed columns
marker_data=lat_lon_grp.reset_index()
marker_data.head()

,Lat,Lng,City,Humidity
0,-54.80,-68.30,Ushuaia,1
1,-53.15,-70.92,Punta Arenas,1
2,-46.60,168.33,Bluff,1
3,-46.28,169.85,Kaitangata,1
4,-46.19,168.86,Mataura,1


In [10]:
#Pulling Latitude and longitude variables into a new DataFrame to create markers for the map
locations = marker_data[['Lat','Lng']]
locations.head()

,Lat,Lng
0,-54.80,-68.30
1,-53.15,-70.92
2,-46.60,168.33
3,-46.28,169.85
4,-46.19,168.86


In [11]:
#Turning location DataFrame into a list so Latitude and Longitude can be grouped together for analysis in next steps
locationlist = locations.values.tolist()
#len(locationlist)
locationlist

[[-54.8, -68.3],
 [-53.15, -70.92],
 [-46.6, 168.33],
 [-46.28, 169.85],
 [-46.19, 168.86],
 [-46.13, 167.68],
 [-45.87, 170.5],
 [-45.58, -72.07],
 [-45.42, 167.72],
 [-42.88, 147.33],
 [-42.78, 147.06],
 [-41.87, -73.82],
 [-41.41, 175.52],
 [-41.15, -71.31],
 [-39.03, -67.58],
 [-38.55, -58.74],
 [-38.27, 141.97],
 [-38.0, -57.56],
 [-37.83, 140.77],
 [-37.62, -73.65],
 [-37.2, 145.07],
 [-37.07, 144.22],
 [-35.35, 150.47],
 [-35.33, -72.42],
 [-35.17, 173.17],
 [-34.68, 150.87],
 [-34.53, 20.04],
 [-34.48, -54.33],
 [-34.42, 19.23],
 [-33.93, 18.42],
 [-33.92, 25.57],
 [-33.87, 121.9],
 [-33.7, -53.46],
 [-33.65, 115.33],
 [-33.59, 26.89],
 [-33.52, -53.37],
 [-33.03, 137.58],
 [-33.02, 27.91],
 [-33.01, 17.94],
 [-32.72, 152.15],
 [-32.5, 137.77],
 [-32.25, 115.77],
 [-31.54, -68.54],
 [-31.41, -64.18],
 [-30.08, -51.73],
 [-29.95, -71.34],
 [-29.86, 31.03],
 [-29.68, -53.81],
 [-29.14, -59.26],
 [-28.45, 18.73],
 [-28.24, -48.67],
 [-27.18, 151.27],
 [-26.82, -65.22],
 [-26.62, -

In [12]:
#Saving city names to a list to be used in map in the next steps
labels = marker_data['City'].values.tolist()
labels

['Ushuaia',
 'Punta Arenas',
 'Bluff',
 'Kaitangata',
 'Mataura',
 'Tuatapere',
 'Dunedin',
 'Coyhaique',
 'Te Anau',
 'Hobart',
 'New Norfolk',
 'Ancud',
 'Waipawa',
 'Bariloche',
 'General Roca',
 'Necochea',
 'Codrington',
 'Mar del Plata',
 'Mount Gambier',
 'Lebu',
 'Broadford',
 'Castlemaine',
 'Ulladulla',
 'Constitución',
 'Ahipara',
 'Kiama',
 'Bredasdorp',
 'Rocha',
 'Hermanus',
 'Cape Town',
 'Port Elizabeth',
 'Esperance',
 'Chui',
 'Busselton',
 'Port Alfred',
 'Santa Vitória do Palmar',
 'Whyalla',
 'East London',
 'Saldanha',
 'Nelson Bay',
 'Port Augusta',
 'Kwinana',
 'San Juan',
 'Córdoba',
 'Arroio dos Ratos',
 'Coquimbo',
 'Durban',
 'Santa Maria',
 'Goya',
 'Warmbad',
 'Imbituba',
 'Dalby',
 'San Miguel de Tucumán',
 'San Vicente',
 'Taltal',
 'Hervey Bay',
 'Yulara',
 'Beloha',
 'Xai-Xai',
 'Carnarvon',
 'Castro',
 'Iguape',
 'Mariental',
 'Chokwé',
 'Dutlwe',
 'Alice Springs',
 'Antofagasta',
 'Emerald',
 'Manyana',
 'Rikitea',
 'Arraial do Cabo',
 'Terra Rica',


In [13]:
#Install necessary extension for maps
#!jupyter nbextension enable --py gmaps

In [14]:
#Setting wight for map based on the Humidity column. The color range from red to green represents humidity intensity
weights=marker_data['Humidity']

# Plotting Heatmap
# map types ['ROADMAP', 'HYBRID', 'TERRAIN', 'SATELLITE']
fig = gmaps.figure(map_type='ROADMAP',center=(5.0, -5.0), zoom_level=2)

# Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights,max_intensity=5,point_radius = 15)

# Adding heat layer to base map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
#documentation -- https://jupyter-gmaps.readthedocs.io/en/latest/api.html
#import widget that allows map to be exported to html

Map was not exported to HMTL here in order to protect google key but is saved to PNG file that can be viewed in the Images folder

### Creating a new DataFrame fitting weather criteria
* Narrowing down the cities to fit personally selected weather conditions.
* Dropping any rows will null values.

#### Selecting ideal conditions for a vacation spot
* A max temperature higher than 30 but lower than 35 degrees Celsius.
* Wind speed less than 10 mph.
* Zero cloudiness.

In [16]:
#Selecting ideals conditions based on the criteria above
ideal_weather=weather.loc[(weather["Max Temp"]>=30) & (weather["Max Temp"]<=35) & \
                          (weather["Wind Speed"]<=4) & (weather["Cloudiness"]<=30) ,:]
#Dropping rows with null values
ideal_weather.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
148,Taoudenni,22.68,-3.98,33.73,14,6,2.57,ML
340,Lakeside,30.13,-81.77,32.00,58,1,2.10,US
562,Honiara,-9.43,159.95,31.00,70,20,1.50,SB


### Hotel Map
* Creating new DataFrame named `hotel_df`.
* Adding a "Hotel Name" column to the DataFrame.
* Setting parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# Creating a new DataFrame called hotel_df and adding the columns "Hotel Name" and "lat_lng"
hotel_df=ideal_weather
hotel_df["Hotel Name"]=""
hotel_df["lat_lng"]=""
hotel_df

C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name,lat_lng
148,Taoudenni,22.68,-3.98,33.73,14,6,2.57,ML,,
340,Lakeside,30.13,-81.77,32.00,58,1,2.10,US,,
562,Honiara,-9.43,159.95,31.00,70,20,1.50,SB,,


In [18]:
#finding keys
curr_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

In [19]:
# Dependencies
import requests
import json

In [20]:
#Creating a for loop to go through each row of data, get the Latitude and Longitude
# and put them together in a new column called lat_lng

for index,row in hotel_df.iterrows():
    #get latitude
    lat=row['Lat']
   
    #get longitude
    lng=row['Lng']
    
    #combine latlng
    latlng=f"{lat},{lng}"
   
    hotel_df.loc[index,"lat_lng"]=latlng

hotel_df

C:\Users\clemi\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name,lat_lng
148,Taoudenni,22.68,-3.98,33.73,14,6,2.57,ML,,"22.68,-3.98"
340,Lakeside,30.13,-81.77,32.00,58,1,2.10,US,,"30.13,-81.77"
562,Honiara,-9.43,159.95,31.00,70,20,1.50,SB,,"-9.43,159.95"


In [21]:
#Setting up parameters for API call to get hotels within 5000 meters of the lat,lng coordinates
base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
distance="5000"
params={"radius":distance,
        "type":"lodging",
        'key':g_key}

In [22]:
#For loop pulling hotels from Google API places based on lat_lng coordinates and entering the name in the hotel_df DataFrame
for index,row in hotel_df.iterrows():
    #get latitude and longitude
    coordinates=row['lat_lng']
    
    city=row['City']
    #add that location into the parameter
    params['location']=coordinates
    
    #printing results
    print("                                                         ")
    print("                                                         ")
    print(f"retreiving results for {city}, lat,lng ({coordinates}) ")
    print("----------------------------------------------------------")
    print("                                                         ")
    
    response =requests.get(base_url,params=params).json()

    #extract results
    results=response['results']
    print(json.dumps(results, indent=4))
    try:
        hotel_df.loc[index,'Hotel Name']=results[0]['name']
        
    except(KeyError,IndexError):
        hotel_df.loc[index,'Hotel Name']= "No hotels in the area"
#results

                                                         
                                                         
retreiving results for Taoudenni, lat,lng (22.68,-3.98) 
----------------------------------------------------------
                                                         
[]
                                                         
                                                         
retreiving results for Lakeside, lat,lng (30.13,-81.77) 
----------------------------------------------------------
                                                         
[
    {
        "business_status": "OPERATIONAL",
        "geometry": {
            "location": {
                "lat": 30.1006245,
                "lng": -81.7455961
            },
            "viewport": {
                "northeast": {
                    "lat": 30.1018376302915,
                    "lng": -81.7441887697085
                },
                "southwest": {
                    "lat": 30.0991396

[
    {
        "business_status": "OPERATIONAL",
        "geometry": {
            "location": {
                "lat": -9.429483,
                "lng": 159.952769
            },
            "viewport": {
                "northeast": {
                    "lat": -9.428257819708497,
                    "lng": 159.9542691802915
                },
                "southwest": {
                    "lat": -9.430955780291502,
                    "lng": 159.9515712197085
                }
            }
        },
        "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
        "id": "95ddcbf1ab20a3922ffe0f53e831d50668121780",
        "name": "Heritage Park Hotel",
        "opening_hours": {
            "open_now": true
        },
        "photos": [
            {
                "height": 4032,
                "html_attributions": [
                    "<a href=\"https://maps.google.com/maps/contrib/102787980935807797600\">Heritage Park Hotel</a>"
               

In [23]:
#Cgreating new DataFrame to hold essential information
hotel_name_df=hotel_df[["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Hotel Name"]]

In [24]:
hotel_name_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
148,Taoudenni,22.68,-3.98,33.73,14,6,2.57,ML,No hotels in the area
340,Lakeside,30.13,-81.77,32.00,58,1,2.10,US,Whitey's Fish Camp
562,Honiara,-9.43,159.95,31.00,70,20,1.50,SB,Heritage Park Hotel


In [25]:
# Adding marker layer on top of heat map
map_data=hotel_df[["Lat","Lng","Hotel Name","City","Country" ]]
hotel_coordinates = map_data[['Lat','Lng']]

locationlist = hotel_coordinates.values.tolist()

#Setting up info box to show Hotel Name, City and Country when a ticker is selected
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_marker=gmaps.marker_layer(locationlist,info_box_content=hotel_info)

fig.add_layer(hotel_marker)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))